In [1]:
!pip install transformers accelerate bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
from google.colab import userdata
secret_key = userdata.get('HF_TOKEN')

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=bnb_config,  token=secret_key)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, token=secret_key)
# model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

# output = model.generate(**model_inputs)

# print(tokenizer.decode(output[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
import torch

def get_log_prob_of_completion(
        model,
        tokenizer,
        prompt,
        completion,
        device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
):
        """
        Convenience function for computing the log probability of a completion
        given a prompt.
        """
        # tokenize the prompt and the completion
        # truncate so as to fit into to maximal context window of llama2
        # which is 2048 tokens
        input_ids = tokenizer(
                prompt + completion,
                return_tensors='pt',
                truncation=True,
                max_length=2048,
        )['input_ids'].to(device)

        # separately tokenize prompt
        # so as to access the logits for the completion only
        # when scoring the completion
        input_ids_prompt = tokenizer(
                prompt,
                return_tensors='pt',
                truncation=True,
                max_length=2048
        )['input_ids'].to(device)

        # create attention mask and position ids
        attention_mask = (input_ids != tokenizer.eos_token_id).to(dtype=torch.int64)
        position_ids = attention_mask.cumsum(-1)-1
        # get the logits for the completion
        with torch.no_grad():
                out = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        position_ids=position_ids
                )

        # get the logits of the completion
        # for that, make a tensor of the logits
        # for the completion only
        # in particular, we shift the indices by one to the left to access logits of the
        # actual sequence tokens
        logits_completion = out.logits[:, :-1]
        logits_completion = logits_completion.squeeze()
        # get the log probabilities for the completion
        log_probs = torch.nn.functional.log_softmax(
                logits_completion,
                dim=-1
        )
        # retrieve the logit corresponding to the actual completion tokens
        try:
                log_completion_tokens = log_probs.gather(
                        dim=-1,
                        index=input_ids[:, 1:].squeeze().unsqueeze(-1)
                )
        except:
                log_completion_tokens = log_probs.gather(
                        dim=-1,
                        index=input_ids[:, 1:].unsqueeze(-1)
                )

        continuationConditionalLogProbs = log_completion_tokens[
                (input_ids_prompt.shape[-1]-1):
        ]
        completion_log_prob = torch.mean(
                continuationConditionalLogProbs
        ).cpu()

        return completion_log_prob

In [5]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/coqa")

import pandas as pd

dataset.set_format(type="pandas")
df = dataset["validation"][:]
df.head()

Generating train split:   0%|          | 0/7199 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

,source,story,questions,answers
0,mctest,"Once upon a time, in a barn near a farm house,...","[What color was Cotton?, Where did she live?, ...","{'input_text': ['white', 'in a barn', 'no', 'w..."
1,mctest,Once there was a beautiful fish named Asta. As...,"[what was the name of the fish, What looked li...","{'input_text': ['Asta.', 'a bottle', 'Asta.', ..."
2,race,"My doorbell rings. On the step, I find the eld...","[Who is at the door?, Is she carrying somethin...",{'input_text': ['An elderly Chinese lady and a...
3,cnn,"(CNN) -- Dennis Farina, the dapper, mustachioe...","[Is someone in showbiz?, Whom?, What did he do...","{'input_text': ['Yes.', 'Dennis Farina', 'Acto..."
4,mctest,Kendra and Quinton travel to and from school e...,[Where do Quinton and Kendra travel to and fro...,"{'input_text': ['school', 'No', 'go to Quentin..."


In [6]:
def make_prompt(df, index, turn):
  prompt = df["story"][index] + "\n\n\n\n"
  for i in range(turn + 1):
    if i > 0:
      prompt = prompt + df["answers"][index]["input_text"][i - 1] + "\n\n"
    prompt = prompt + df["questions"][index][i] + "\n\n"
  return prompt

In [7]:
prompt_df = pd.DataFrame()

for index, row in df.iterrows():
  for turn, question in enumerate(row["questions"]):
    prompt = make_prompt(df, index, turn)
    completion = row["answers"]["input_text"][turn]
    data = {
        "prompt": [prompt],
        "completion": [completion]
    }
    temp_df = pd.DataFrame(data)
    prompt_df = pd.concat([prompt_df, temp_df], ignore_index=True)

In [8]:
def make_n_examples(n, df):
  header = "You are supposed to answer the last question in a conversation about a topic. Here are some samples for how a conversation should look like:\n\n\n"
  body = ""
  for index in range(1, n * 10 + 1, 10):
    prompt = df["prompt"].iloc[-index].strip()
    prompt = prompt.splitlines()
    prompt = prompt[:-1]
    prompt = "\n".join(prompt)
    body = body + prompt + "\n\n\n"
  footer = "Now it is your turn to answer and please keep it succinct.\n\n\n"
  return header + body + footer

In [9]:
example = make_n_examples(3, prompt_df)

In [10]:
print(example)

You are supposed to answer the last question in a conversation about a topic. Here are some samples for how a conversation should look like:


Las Vegas (, Spanish for "The Meadows"), officially the City of Las Vegas and often known simply as Vegas, is the 28th-most populated city in the United States, the most populated city in the state of Nevada, and the county seat of Clark County. The city anchors the Las Vegas Valley metropolitan area and is the largest city within the greater Mojave Desert. Las Vegas is an internationally renowned major resort city, known primarily for its gambling, shopping, fine dining, entertainment, and nightlife. The Las Vegas Valley as a whole serves as the leading financial, commercial, and cultural center for Nevada. 

The city bills itself as The Entertainment Capital of the World, and is famous for its mega casino–hotels and associated activities. It is a top three destination in the United States for business conventions and a global leader in the hos

In [11]:
prompt_df.head(15)

,prompt,completion
0,"Once upon a time, in a barn near a farm house,...",white
1,"Once upon a time, in a barn near a farm house,...",in a barn
2,"Once upon a time, in a barn near a farm house,...",no
3,"Once upon a time, in a barn near a farm house,...",with her mommy and 5 sisters
4,"Once upon a time, in a barn near a farm house,...",orange and white
5,"Once upon a time, in a barn near a farm house,...",no
6,"Once upon a time, in a barn near a farm house,...",she painted herself
7,"Once upon a time, in a barn near a farm house,...",the farmer
8,"Once upon a time, in a barn near a farm house,...",they started laughing
9,"Once upon a time, in a barn near a farm house,...",a bucket of water


In [12]:
prompt_df["prompt"] = prompt_df["prompt"].apply(lambda x: example + x)

In [13]:
prompt_df.head(15)

,prompt,completion
0,You are supposed to answer the last question i...,white
1,You are supposed to answer the last question i...,in a barn
2,You are supposed to answer the last question i...,no
3,You are supposed to answer the last question i...,with her mommy and 5 sisters
4,You are supposed to answer the last question i...,orange and white
5,You are supposed to answer the last question i...,no
6,You are supposed to answer the last question i...,she painted herself
7,You are supposed to answer the last question i...,the farmer
8,You are supposed to answer the last question i...,they started laughing
9,You are supposed to answer the last question i...,a bucket of water


In [14]:
prompt_df = prompt_df.head(100)

In [15]:
log_prob_list = []

for _, row in prompt_df.iterrows():
  log_prob = get_log_prob_of_completion(model, tokenizer, row["prompt"], row["completion"])
  log_prob_list.append(log_prob)

In [16]:
import math

len([1 for x in log_prob_list if math.isnan(x)])

9

In [17]:
prompt_df["log_prob"] = [x.item() for x in log_prob_list]
prompt_df.head(15)

,prompt,completion,log_prob
0,You are supposed to answer the last question i...,white,-0.181614
1,You are supposed to answer the last question i...,in a barn,-0.347703
2,You are supposed to answer the last question i...,no,-0.005833
3,You are supposed to answer the last question i...,with her mommy and 5 sisters,-1.183194
4,You are supposed to answer the last question i...,orange and white,-1.201918
5,You are supposed to answer the last question i...,no,-0.018885
6,You are supposed to answer the last question i...,she painted herself,-0.516590
7,You are supposed to answer the last question i...,the farmer,-0.427319
8,You are supposed to answer the last question i...,they started laughing,-0.360457
9,You are supposed to answer the last question i...,a bucket of water,-0.637614


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
prompt_df.to_csv("/content/drive/MyDrive/RL_for_LLMs/coqa_13b_chat_few_shot_small.csv")